# Dash Flight Details


In [1]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

In [2]:
# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [3]:
print(airline_data.shape)
airline_data.head()

(27000, 110)


,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [5]:
# Build dash app layout
app.layout = html.Div(children=[html.H1('Flight Delay Time Statistics',
                                         style={'textAlign':'center',
                                               'color':'#503D36',
                                               'font-size':30}),
                                html.Div(["Input Year: ", 
                                          dcc.Input(id='input-year', 
                                                    value='2010',
                                                    type='number',
                                                    style={'height':'35px',
                                                           'font-size':30},)],
                                         style={'font-size': 30},
                                        ),
                                html.Br(),
                                html.Br(), 
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),

                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),

                                html.Div(dcc.Graph(id='late-plot'),
                                                   style={'width':'65%'})
                                ])


In [6]:
def compute_info(airline_data, entered_year):
    """ Compute_info function description

    This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

    Arguments:
        airline_data: Input airline data.
        entered_year: Input year for which computation needs to be performed.

    Returns:
        Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

    """
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

In [7]:
# Callback decorator
@app.callback([Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')],
               [Input(component_id='input-year', component_property='value')]
            )
# Computation to callback function and return graph
def get_graph(entered_year):

    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)

    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay',
                          color='Reporting_Airline',
                          title='Average carrier delay time (minutes) by airline')
    # carrier_fig = go.Figure(data=go.Scatter(x=avg_car['Month'], 
    #                                         y=avg_car['CarrierDelay'],
    #                                         mode='lines', 
    #                                         marker=dict(color='green'))) 
    # carrier_fig.update_layout(title='Average carrier delay time (minutes) by airline',
    #                           xaxis_title='Month', yaxis_title='CarrierDelay')                       
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay',
                          color='Reporting_Airline',
                          title='Average carrier delay time (minutes) by airline')
    # weather_fig = go.Figure(data=go.Scatter(x=avg_weather['Month'], 
    #                                         y=avg_weather['WeatherDelay'],
    #                                         mode='lines', 
    #                                         marker=dict(color='green')))
    # weather_fig.update_layout(title='Average carrier delay time (minutes) by airline',
    #                           xaxis_title='Month', yaxis_title='WeatherDelay')                                            
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay',
                         color='Reporting_Airline',
                         title='Average carrier delay time (minutes) by airline')
    # nas_fig = go.Figure(data=go.Scatter(x=avg_NAS['Month'], 
    #                                         y=avg_NAS['NASDelay'],
    #                                         mode='lines', 
    #                                         marker=dict(color='green')))
    # nas_fig.update_layout(title='Average carrier delay time (minutes) by airline',
    #                           xaxis_title='Month', yaxis_title='NASDelay')                                                                       
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay',
                          color='Reporting_Airline',
                          title='Average carrier delay time (minutes) by airline')
    # sec_fig = go.Figure(data=go.Scatter(x=avg_sec['Month'], 
    #                                         y=avg_sec['SecurityDelay'],
    #                                         mode='lines', 
    #                                         marker=dict(color='green')))     
    # sec_fig.update_layout(title='Average carrier delay time (minutes) by airline',
    #                           xaxis_title='Month', yaxis_title='SecurityDelay')                                                                 
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay',
                          color='Reporting_Airline',
                          title='Average carrier delay time (minutes) by airline')
    # late_fig = go.Figure(data=go.Scatter(x=avg_late['Month'], 
    #                                         y=avg_late['LateAircraftDelay'],
    #                                         mode='lines', 
    #                                         marker=dict(color='green'))) 
    # late_fig.update_layout(title='Average carrier delay time (minutes) by airline',
    #                           xaxis_title='Month', yaxis_title='LateAircraftDelay')                                                                       

    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

In [8]:
# # Uncomment to run the application                   
# if __name__ == '__main__':
#     app.run_server()